In [ ]:
import retrieval.cache as cache
import signals.ema as ema
import signals.simple_scalping as simple_scalping

import numpy as np
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go

from backtesting import Backtest, Strategy
from tqdm import tqdm

### Overarching Constants

In [10]:
ticker = 'BTC-USD'
start = '2023-01-01'
end = '2023-12-31'
interval = '1d'

columns_open = 'Open'
columns_high = 'High'
columns_low = 'Low'
columns_close = 'Close'
columns_volume = 'Volume'
columns_rsi = 'RSI'
columns_ema_slow = 'EMA Slow' # Need to change function calls below if change this one
columns_ema_fast = 'EMA Fast' # Need to change function calls below if change this one
columns_atr = 'ATR'
columns_simple_scalp = 'Simple Scalp'

### Make sure we have some data to work with

In [11]:
if not cache.exist_sql_db(ticker, interval):
    cache.cache_ticker(ticker, interval, start, end)

df = cache.load_ticker(ticker, interval, start, end)

#df

### Add TA Columns

In [ ]:
df[columns_ema_slow] = ta.ema(df[columns_close], length=50)
df[columns_ema_fast] = ta.ema(df[columns_close], length=30)
df[columns_rsi] = ta.rsi(df[columns_close], length=10)

# Average True Range (or Volatility) - help to define stop loss / take profit distance
df[columns_atr] = ta.atr(df[columns_high], df[columns_low], df[columns_close], length=7)

# Bollinger bands
df=df.join(ta.bbands(df[columns_close], length=15, std=1.5))

#df


### EMA signal function

In [13]:
# Needed for progress_apply
tqdm.pandas()

df.reset_index(inplace=True)

# row.name is the index of the row -> 'Date' column
df['EMA Signal'] = df.progress_apply(
  lambda row: ema.two_above_or_below(df, row.name, 7) if row.name >= 20 else 0, 
  axis=1
)

#df

100%|██████████| 364/364 [00:00<00:00, 3710.76it/s]


### Calculate the Simple Scalping Signal

In [14]:
df[columns_simple_scalp] = df.progress_apply(
  lambda row: simple_scalping.simple_signal(df, row.name, 7), axis=1
)

#df[df['Simple Scalp'] != 0].head(20)

100%|██████████| 364/364 [00:00<00:00, 1886.03it/s]


,index,Date,Open,High,Low,Close,Adj Close,Volume,EMA Slow,EMA Fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMA Signal,Simple Scalp
61,61,2023-03-03,23476.632812,23479.347656,22213.238281,22362.679688,22362.679688,26062404610,22175.811119,23010.621345,37.599846,791.360143,22798.022444,23801.733073,24805.443702,8.433929,-0.216867,2,2
62,62,2023-03-04,22362.923828,22405.177734,22198.980469,22353.349609,22353.349609,11166012913,22182.773413,22968.216716,37.512441,707.759538,22565.377542,23654.249609,24743.121676,9.206566,-0.097361,2,2
66,66,2023-03-08,22216.441406,22268.896484,21708.050781,21718.080078,21718.080078,22536575684,22183.675443,22783.853715,30.919373,579.478755,21975.800010,23025.496224,24075.192438,9.117686,-0.122759,2,2
67,67,2023-03-09,21720.080078,21802.716797,20210.306641,20363.021484,20363.021484,30364664171,22112.277249,22627.670990,21.407519,724.187973,21422.924385,22770.441406,24117.958427,11.835669,-0.393280,2,2
68,68,2023-03-10,20367.001953,20370.595703,19628.253906,20187.244141,20187.244141,39578257695,22036.785754,22470.224097,20.498609,726.781449,20948.921991,22519.758203,24090.594415,13.950738,-0.242443,2,2
69,69,2023-03-11,20187.876953,20792.525391,20068.660156,20632.410156,20632.410156,30180288176,21981.712202,22351.655455,28.983228,726.364837,20655.493906,22348.710417,24041.926927,15.152700,-0.006817,2,2
110,110,2023-04-21,28249.230469,28349.968750,27177.365234,27276.910156,27276.910156,20759504330,27234.187273,28337.715194,35.085429,1010.550379,27700.823531,29329.372656,30957.921782,11.105244,-0.130151,2,2
112,112,2023-04-23,27816.144531,27820.244141,27400.314453,27591.384766,27591.384766,12785446832,27270.172967,28258.167881,39.365005,888.464385,27606.000569,29298.388021,30990.775473,11.552768,-0.004318,2,2
127,127,2023-05-08,28450.457031,28663.271484,27310.134766,27694.273438,27694.273438,19122903752,27924.637380,28547.120658,39.269871,1044.934575,27809.033312,28720.091146,29631.148979,6.344394,-0.062982,2,2
128,128,2023-05-09,27695.068359,27821.400391,27375.601562,27658.775391,27658.775391,14128593256,27914.211419,28489.808060,38.971603,959.343754,27843.158201,28728.986849,29614.815497,6.166794,-0.104074,2,2


### Create Dots for Simple Scalping Signals

In [15]:
def render_short_long_signal_point(row: pd.Series, column_name:str) -> str:
  if row[column_name] == simple_scalping.LONG_SIGNAL:
    return row[columns_low]-1e-3
  elif row[column_name] == simple_scalping.SHORT_SIGNAL:
    return row[columns_high]+1e-3
  
  return np.nan

df['ss_point'] = df.apply(lambda row: render_short_long_signal_point(row, columns_simple_scalp), axis=1)

### Plot the Graph

In [16]:
partition = df

fig = go.Figure(data=[go.Candlestick(x=partition.index,
                open=partition[columns_open],
                high=partition[columns_high],
                low=partition[columns_low],
                close=partition[columns_close]),
                
                go.Scatter(x=partition.index, y=partition['BBL_15_1.5'], 
                           line=dict(color='blue', width=1), 
                           name='BBL'),

                go.Scatter(x=partition.index, y=partition['BBU_15_1.5'],
                            line=dict(color='blue', width=1),
                            name='BBU'),

                go.Scatter(x=partition.index, y=partition[columns_ema_slow],
                            line=dict(color='orange', width=1),
                            name='EMA Slow'),

                go.Scatter(x=partition.index, y=partition[columns_ema_fast],
                            line=dict(color='green', width=1),
                            name='EMA Fast'),

                go.Scatter(x=partition.index, y=partition[columns_rsi],
                            line=dict(color='red', width=1),
                            name='RSI'),
                ])

fig.add_trace(go.Scatter(x=partition.index, y=partition['ss_point'],
                            mode='markers', marker=dict(color='darkorange', size=5),
                            name=columns_simple_scalp))

### Backtesting

In [17]:
def signal():
    return df[columns_simple_scalp]

class MyStrategy(Strategy):
    # Stop loss coefficient
    sl_coef = 1.1
    tpsl_ratio = 1.5

    def init(self):
        super().init()

        self.simple_scalp_signal = self.I(signal)

    def next(self):
        super().next()

        # Stop-loss distance
        sl_atr = self.sl_coef * self.data.ATR[-1]
        tpsl_ratio = self.tpsl_ratio

        
        if self.simple_scalp_signal == simple_scalping.LONG_SIGNAL and len(self.trades) == 0:
            sl1 = self.data[columns_close][-1] - sl_atr
            tp1 = self.data[columns_close][-1] + sl_atr * tpsl_ratio
            self.buy(sl=sl1, tp=tp1)

        if self.simple_scalp_signal == simple_scalping.SHORT_SIGNAL and len(self.trades) == 0:
            sl1 = self.data[columns_close][-1] + sl_atr
            tp1 = self.data[columns_close][-1] - sl_atr * tpsl_ratio
            self.sell(sl=sl1, tp=tp1)

bt = Backtest(df, MyStrategy, cash=400000, commission=.002)

bt.run()

/var/folders/_w/4mgyjmws5qq_yf18_4s5kff80000gn/T/ipykernel_36798/1876275384.py:32: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                     363.0
Duration                                363.0
Exposure Time [%]                   30.494505
Equity Final [$]                354919.858581
Equity Peak [$]                 428154.810105
Return [%]                         -11.270035
Buy & Hold Return [%]              153.574131
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -19.954834
Avg. Drawdown [%]                   -6.449218
Max. Drawdown Duration                  216.0
Avg. Drawdown Duration                   49.6
# Trades                                 15.0
Win Rate [%]                        33.333333
Best Trade [%]                       5.988683
Worst Trade [%]                     -4.412394
Avg. Trade [%]                    

### Plot the Backtesting Results

In [ ]:
bt.plot()